In [16]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
import glob

# Read in catalog data
catalog = pd.read_csv('observations.txt' , delimiter=' ')
master_catalog = pd.read_csv('master_catalog_jan_2023.csv', delimiter=',')
print(master_catalog)
print('Starting loop...')
# Loop over unique FieldIDs
counter = 0
total = len(catalog['FieldID'].unique())
master_catalog_coords = SkyCoord(ra=master_catalog['RADEG'], dec=master_catalog['DECDEG'], unit=(u.deg, u.deg))
print(len(master_catalog_coords))
for index in range(len(master_catalog_coords)):
    # Find matching file based on FieldID
    idx, d2d, _ = catalog.match_to_catalog_sky(master_catalog_coords[index])
    idx, d2d, _ = master_catalog_coords[index].match_to_catalog_sky(catalog)
    print(catalog.loc[idx])
    break
    file_path = f'Cleaned/c{field_id}p.ascd'
    if not glob.glob(file_path):
        print(f'Error: File not found for FieldID {field_id}')
        continue
    # Read in file data
    file_data = pd.read_csv(file_path, delimiter=' ')
    print('+=================+')
    print('Processing : ', field_id, '(', counter, ' / ', total ,')')
    # file_coords = [SkyCoord(ra=ra_str, dec=dec_str, unit=(u.hourangle, u.deg)) for ra_str, dec_str in zip(file_data['RA'], file_data['Dec'])]
    # file_coords = list(map(lambda x: SkyCoord(ra=x[0], dec=x[1], unit=(u.hourangle, u.deg)), zip(file_data['RA'],
    #                                                                                              file_data['Dec'])))
    file_coords = SkyCoord(ra=file_data['RA'], dec=file_data['Dec'], unit=(u.hourangle, u.deg))
    print('File Coords Processed')
    # Loop over catalog data for this FieldID
    for index, row in catalog[catalog['FieldID']==field_id].iterrows():
        # Create SkyCoord object for catalog data
        ra_deg = float(row['RAh']) * 15 + float(row['RAm']) * 0.25 + float(row['RAs']) * 0.00416667
        dec_deg = float(row['DEd']) + (float(row['DEm'])/60) + (float(row['DEs'])/3600)
        catalog_coord = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg)
        # Find the closest match in file data
        idx, d2d, _ = catalog_coord.match_to_catalog_sky(file_coords)
        # Get i and g values for closest match
        i_value = file_data.loc[idx, 'i']
        g_value = file_data.loc[idx, 'g']
        # Add i and g values to catalog
        catalog.at[index, 'i'] = i_value
        catalog.at[index, 'g'] = g_value
    print('Matched Coords.')
    break
    # catalog.to_csv('outputs2/catalog_updated' + str(counter) +  '.csv', index=False)
    # try:
    #     # catalog.to_csv('outputs2/catalog_updated.csv', index=False)
    #     pass
    # except:
    #     pass
    counter += 1
# Write out updated catalog
print('Done!')
# catalog.to_csv('catalog2_updated.csv', index=False)

             ID       RADEG     DECDEG  CLASS                 DATATABLE
0            C1    0.157917  32.417778    2.0  di_tullio_zinn_13_table2
1         2M034    0.508375  51.606722    4.0                  m31rbcV5
2     PAndAS-03    0.985042  40.888667    1.0            huxor14_table1
3     PAndAS-04    1.178875  47.361797    1.0            huxor14_table1
4     PAndAS-05    1.350625  43.926583    1.0            huxor14_table1
...         ...         ...        ...    ...                       ...
2663        C74  356.707917  45.247222    2.0  di_tullio_zinn_13_table2
2664        C75  357.170417  39.629167    2.0  di_tullio_zinn_13_table2
2665     SDSS18  357.290417  40.458333    4.0  di_tullio_zinn_13_table1
2666  PAndAS-01  359.300125  43.552300    1.0            huxor14_table1
2667  PAndAS-02  359.482042  41.780347    1.0            huxor14_table1

[2668 rows x 5 columns]
Starting loop...
2668


AttributeError: 'DataFrame' object has no attribute 'match_to_catalog_sky'

In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
import glob

# Read in catalog data
catalog = pd.read_csv('observations.txt' , delimiter=' ')
master_catalog = pd.read_csv('master_catalog_jan_2023.csv', delimiter=',')
print(master_catalog)
print('Starting loop...')

# Loop over unique FieldIDs
counter = 0
master_catalog_coords = SkyCoord(ra=master_catalog['RADEG'], dec=master_catalog['DECDEG'], unit=(u.deg, u.deg))
total = len(master_catalog_coords)
ra_deg = (catalog['RAh']) * 15 + (catalog['RAm']) * 0.25 + (catalog['RAs']) * 0.00416667
dec_deg = (catalog['DEd']) + ((catalog['DEm'])/60) + ((catalog['DEs'])/3600)
catalog_coord = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg)
# print(len(master_catalog_coords))
for index in range(total):
    # Find matching file based on FieldID
    idx, d2d, _ = master_catalog_coords[index].match_to_catalog_sky(catalog_coord)
    # print(catalog.loc[idx])
    field_id = catalog.loc[idx]['FieldID']
    file_path = f'Cleaned/c{field_id}p.ascd'
    if not glob.glob(file_path):
        print(f'Error: File not found for FieldID {field_id}')
        continue
    # Read in file data
    file_data = pd.read_csv(file_path, delimiter=' ')
    print('+=================+')
    print('Processing : ', field_id, '(', index, ' / ', total ,')')
    file_coords = SkyCoord(ra=file_data['RA'], dec=file_data['Dec'], unit=(u.hourangle, u.deg))
    idx, _, _ = master_catalog_coords[index].match_to_catalog_sky(file_coords)
    # print(file_coords.loc[idx])
    i_value = file_data.loc[idx, 'i']
    g_value = file_data.loc[idx, 'g']
    # Add i and g values to catalog
    master_catalog.at[index, 'i'] = i_value
    master_catalog.at[index, 'g'] = g_value
    print('Coords Processed')
    master_catalog.to_csv('master_updated2.csv', index=False)
# Write out updated catalog
print('Outputing to csv')
master_catalog.to_csv('master_updated.csv', index=False)
print('Done!')

             ID       RADEG     DECDEG  CLASS                 DATATABLE
0            C1    0.157917  32.417778    2.0  di_tullio_zinn_13_table2
1         2M034    0.508375  51.606722    4.0                  m31rbcV5
2     PAndAS-03    0.985042  40.888667    1.0            huxor14_table1
3     PAndAS-04    1.178875  47.361797    1.0            huxor14_table1
4     PAndAS-05    1.350625  43.926583    1.0            huxor14_table1
...         ...         ...        ...    ...                       ...
2663        C74  356.707917  45.247222    2.0  di_tullio_zinn_13_table2
2664        C75  357.170417  39.629167    2.0  di_tullio_zinn_13_table2
2665     SDSS18  357.290417  40.458333    4.0  di_tullio_zinn_13_table1
2666  PAndAS-01  359.300125  43.552300    1.0            huxor14_table1
2667  PAndAS-02  359.482042  41.780347    1.0            huxor14_table1

[2668 rows x 5 columns]
Starting loop...
+=================+
Processing :  m095 ( 0  /  2668 )
Coords Processed
+=================+
Pro

find how close from table 2, g, i if either greater than 23 delete

Go through line by line in master_catalog and find closest coord match in table 2 then use that to find field and
then we can find i and g values

In [3]:
import pandas as pd
from astropy.coordinates import SkyCoord
from astropy import units as u
import glob

# Read in catalog data
catalog = pd.read_csv('observations.txt' , delimiter=' ')
master_catalog = pd.read_csv('master_updated2.csv', delimiter=',')
print(master_catalog)
print('Starting loop...')

# Loop over unique FieldIDs
counter = 0
master_catalog_coords = SkyCoord(ra=master_catalog['RADEG'], dec=master_catalog['DECDEG'], unit=(u.deg, u.deg))
total = len(master_catalog_coords)
ra_deg = (catalog['RAh']) * 15 + (catalog['RAm']) * 0.25 + (catalog['RAs']) * 0.00416667
dec_deg = (catalog['DEd']) + ((catalog['DEm'])/60) + ((catalog['DEs'])/3600)
catalog_coord = SkyCoord(ra=ra_deg * u.deg, dec=dec_deg * u.deg)
# print(len(master_catalog_coords))
for index in range(2400, total):
    # Find matching file based on FieldID
    idx, d2d, _ = master_catalog_coords[index].match_to_catalog_sky(catalog_coord)
    # print(catalog.loc[idx])
    field_id = catalog.loc[idx]['FieldID']
    file_path = f'Cleaned/c{field_id}p.ascd'
    if not glob.glob(file_path):
        print(f'Error: File not found for FieldID {field_id}')
        continue
    # Read in file data
    file_data = pd.read_csv(file_path, delimiter=' ')
    print('+=================+')
    print('Processing : ', field_id, '(', index, ' / ', total ,')')
    file_coords = SkyCoord(ra=file_data['RA'], dec=file_data['Dec'], unit=(u.hourangle, u.deg))
    idx, _, _ = master_catalog_coords[index].match_to_catalog_sky(file_coords)
    # print(file_coords.loc[idx])
    i_value = file_data.loc[idx, 'i']
    g_value = file_data.loc[idx, 'g']
    # Add i and g values to catalog
    master_catalog.at[index, 'i'] = i_value
    master_catalog.at[index, 'g'] = g_value
    print('Coords Processed')
    master_catalog.to_csv('master_updated3.csv', index=False)
# Write out updated catalog
print('Outputing to csv')
master_catalog.to_csv('maste*r_updated.csv', index=False)
print('Done!')

             ID       RADEG     DECDEG  CLASS                 DATATABLE  \
0            C1    0.157917  32.417778    2.0  di_tullio_zinn_13_table2   
1         2M034    0.508375  51.606722    4.0                  m31rbcV5   
2     PAndAS-03    0.985042  40.888667    1.0            huxor14_table1   
3     PAndAS-04    1.178875  47.361797    1.0            huxor14_table1   
4     PAndAS-05    1.350625  43.926583    1.0            huxor14_table1   
...         ...         ...        ...    ...                       ...   
2663        C74  356.707917  45.247222    2.0  di_tullio_zinn_13_table2   
2664        C75  357.170417  39.629167    2.0  di_tullio_zinn_13_table2   
2665     SDSS18  357.290417  40.458333    4.0  di_tullio_zinn_13_table1   
2666  PAndAS-01  359.300125  43.552300    1.0            huxor14_table1   
2667  PAndAS-02  359.482042  41.780347    1.0            huxor14_table1   

           i       g  
0     22.596  24.741  
1     24.960  25.493  
2     22.599  23.526  
3     1


KeyboardInterrupt

